In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from math import gcd
from fractions import Fraction
import time

def qft_dagger(n):
    """n-qubit QFT†."""
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    return qc

def c_amod15(a, power):
    """Controlled multiplication by a mod 15."""
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    return U

def shor_circuit(N, a):
    n = N.bit_length()
    
    qc = QuantumCircuit(2*n+4, n)
    
    # Initialize counting qubits in superposition
    for q in range(2*n):
        qc.h(q)
    
    # Initialize auxiliary qubits
    qc.x(2*n)
    
    # Apply controlled U operations
    for i in range(2*n):
        qc.append(c_amod15(a, 2**i).control(1), [i] + list(range(2*n, 2*n+4)))
    
    # Apply inverse QFT
    qc.append(qft_dagger(2*n), range(2*n))
    
    # Measure
    qc.measure(range(n), range(n))
    
    return qc

def get_factors(N, a, measured_phases):
    """Get factors from the measured phases."""
    factors = []
    for phase in measured_phases:
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator
        if r % 2 == 0:
            guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
            for guess in guesses:
                print("guess", guess)
                if guess not in [1, N] and (N % guess) == 0:
                    factors.append(guess)
    return factors

def run_shor(N, attempts=10):
    """Run Shor's algorithm to factor N."""
    for _ in range(attempts):
        a = np.random.randint(2, N)
        if gcd(a, N) != 1:
            return [gcd(a, N), N // gcd(a, N)]
        
        qc = shor_circuit(N, a)
        
        simulator = AerSimulator()
        transpiled_qc = transpile(qc, simulator)
        job = simulator.run(transpiled_qc, shots=1000)
        result = job.result()
        counts = result.get_counts()
        
        measured_phases = [int(outcome, 2)/(2**qc.num_clbits) for outcome in counts.keys()]
        factors = get_factors(N, a, measured_phases)
        
        if factors:
            return factors
    
    return None

# Run the algorithm
N = 123
timeout = 60  # Timeout in seconds
start_time = time.time()

while True:
    if time.time() - start_time > timeout:
        print(f"Timeout reached after {timeout} seconds. Stopping execution.")
        break

    factors = run_shor(N)
    if factors:
        print(f"Factors of {N}: {factors}")
        break
    else:
        print(f"Failed to factor {N} after multiple attempts. Trying again...")

    # Optional: Add a small delay to prevent excessive CPU usage
    time.sleep(0.1)